In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge, LinearRegression, Lasso, LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures, OneHotEncoder
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from warnings import filterwarnings
filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
rs = 42

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [2]:
! git clone https://github.com/dsreload/ml_2
data = pd.read_csv("ml_2/boston.csv")
x = data.iloc[:, :13]
y = data.iloc[:, 13:]

Cloning into 'ml_2'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 8 (delta 1), reused 4 (delta 1), pack-reused 3 (from 1)
Receiving objects: 100% (8/8), 8.99 MiB | 7.90 MiB/s, done.
Resolving deltas: 100% (1/1), done.


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%, предварительно выделив целевую переменную (колонка 'MEDV').

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

2. Обучите стандартную регрессию, а также Ridge и  Lasso с параметрами по умолчанию и выведите их R2 на тестовой выборке

In [ ]:
for model in [LinearRegression(), Ridge(random_state=rs), Lasso(random_state=rs)]:
    fit_model = model.fit(x_train, y_train)
    print(type(model).__name__,"R^2:", model.score(x_test, y_test))

LinearRegression R^2: 0.7013157225661428
Ridge R^2: 0.6972742919686512
Lasso R^2: 0.6208389302645347


3. Для Ridge и Lasso подберите коэффициент регуляризации двумя способами 1) GridSearchCV, 2) RidgeCV и LassoCV, в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по всем моделям и сравните с предыдущими результатами.

In [ ]:
for model in [Ridge(random_state=rs), Lasso(random_state=rs)]:
    params = {'alpha': [10**x for x in range(-5, 6)]}
    grid = GridSearchCV(model, params, scoring='r2')
    fit_model = grid.fit(x_train, y_train)
    print(type(model).__name__, "GridSearchCV best params:",
          grid.best_params_,"R^2:", fit_model.score(x_test, y_test)) # R^2 увеличился для обоих

Ridge GridSearchCV best params: {'alpha': 0.1} R^2: 0.701077526427967
Lasso GridSearchCV best params: {'alpha': 0.001} R^2: 0.701156769515765


4. Проведите масштабирование выборки (используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [ ]:
pipe_rs = Pipeline([('std_scaler', StandardScaler()), ('clf', Ridge(random_state=rs))])
pipe_rm = Pipeline([('mm_scaler',  MinMaxScaler()),   ('clf', Ridge(random_state=rs))])
pipe_ls = Pipeline([('std_scaler', StandardScaler()), ('clf', Lasso(random_state=rs))])
pipe_lm = Pipeline([('mm_scaler',  MinMaxScaler()),   ('clf', Lasso(random_state=rs))])
pipes = [{'name': 'Ridge with Standard Scaler', 'clf': pipe_rs}, {'name': 'Ridge with Min Max Scaler',  'clf': pipe_rm},
          {'name': 'Lasso with Standard Scaler', 'clf': pipe_ls}, {'name': 'Lasso with Min Max Scaler',  'clf': pipe_lm}]
for pipe in pipes:
    pipe['clf'].fit(x_train, y_train)
    print(pipe['name'],"R^2:", pipe['clf'].score(x_test, y_test)) # R^2 увеличился для Ridge и уменьшился для Lasso

Ridge with Standard Scaler R^2: 0.7013381968936976
Ridge with Min Max Scaler R^2: 0.6905583182401169
Lasso with Standard Scaler R^2: 0.6262192825686278
Lasso with Min Max Scaler R^2: 0.18006988418558134


5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами.

In [ ]:
params = {'alpha': [10**x for x in range(-5, 6)]}
clfs = [('Ridge', Ridge(random_state=rs), params), ('Lasso', Lasso(random_state=rs), params)]
for clfer, clf, grid in clfs:
    pipes = [('Standard Scaler', Pipeline(steps=[('scaler', StandardScaler()),('classifier', clf)])), ('Min Max Scaler',  Pipeline(steps=[('scaler', MinMaxScaler()),('classifier', clf)]))]
    for scaler, pipe in pipes:
        search = GridSearchCV(pipe, {f'classifier__{name}': value for name, value in grid.items()}, scoring='r2')
        search.fit(x_train, y_train)
        print(f"{clfer} with {scaler} {search.best_params_} R^2:", search.score(x_test, y_test)) # R^2 увеличился значительно для Lasso, для Ridge почти без изменений

Ridge with Standard Scaler {'classifier__alpha': 10} R^2: 0.6998123494501569
Ridge with Min Max Scaler {'classifier__alpha': 0.1} R^2: 0.7005748964221841
Lasso with Standard Scaler {'classifier__alpha': 0.01} R^2: 0.7015446500012794
Lasso with Min Max Scaler {'classifier__alpha': 0.01} R^2: 0.7013247320003582


6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [ ]:
clfs = [('Linear regression', LinearRegression()),('Ridge', Ridge(random_state=rs)), ('Lasso', Lasso(random_state=rs))]
for clf_name, clf in clfs:
    pipes = [('Standard Scaler', Pipeline(steps=[('Scaler', StandardScaler()),('PolynomialFeatures', PolynomialFeatures()),('Classifier', clf)])),
            ('Min Max Scaler', Pipeline(steps=[('Scaler', MinMaxScaler()),('PolynomialFeatures', PolynomialFeatures()),('Classifier', clf)]))]
    for pipe_name, pipe in pipes:
        pipe.fit(x_train, y_train)
        print(f"{clf_name}  {pipe_name} score: ", pipe.score(x_test, y_test)) # R^2 значительно увеличилcя за исключением Lasso с MinMaxScaler

Linear regression  Standard Scaler score:  0.8010149918626801
Linear regression  Min Max Scaler score:  0.8010149918626781
Ridge  Standard Scaler score:  0.8166223235708723
Ridge  Min Max Scaler score:  0.7940324407262243
Lasso  Standard Scaler score:  0.7248472388773134
Lasso  Min Max Scaler score:  0.16546969049186844


7. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, добавив PolynomialFeatures, посчитайте R2 и сравните с предыдущими результатами.

In [ ]:
pipe = Pipeline(steps=[('scaler', 'passthrough'),('poly', PolynomialFeatures()),('clf', 'passthrough')])

params = [{'scaler': [StandardScaler(), MinMaxScaler()],
           'clf': [Ridge(random_state=rs), Lasso(random_state=rs)],'clf__alpha': [10**x for x in range(-5, 6)]}]

search = GridSearchCV(pipe, params, scoring='r2', n_jobs=-1)
search.fit(x_train, y_train)
print(search.best_params_, search.score(x_test, y_test)) # R^2 незначительно увеличилcя для Ridge

{'clf': Ridge(random_state=42), 'clf__alpha': 10, 'scaler': StandardScaler()} 0.8132662283409828


8. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2.

In [ ]:
pipe = Pipeline(steps=[('scaler', 'passthrough'),('poly', PolynomialFeatures()),('clf', 'passthrough')])

params = [{'scaler': [StandardScaler(), MinMaxScaler()],'poly__degree': [i for i in range(1, 6)],
           'clf': [Ridge(random_state=rs), Lasso(random_state=rs, max_iter=10000, tol=0.035)],'clf__alpha': [10**x for x in range(-5, 6)]}]

search = GridSearchCV(pipe, params, scoring='r2', n_jobs=-1)
search.fit(x_train, y_train)
print(search.best_params_, search.score(x_test, y_test))

{'clf': Ridge(random_state=42), 'clf__alpha': 10, 'poly__degree': 2, 'scaler': StandardScaler()} 0.8132662283409828


http://archive.ics.uci.edu/ml/datasets/Adult

In [3]:
data = pd.read_csv('ml_2/adult.csv', header=None)

9. Разделите выборку на признаки и целевую переменную(колонка class). Замените целевую переменную на числовые значения ('<=50K' - 1, '>50K' - 0).

In [4]:
x = data.loc[:,:13].iloc[1:]
y = data[14].map({'<=50K': 0, '>50K': 1}).iloc[1:]

10. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [ ]:
y.value_counts()

,count
14,
0.0,37155
1.0,11687


In [ ]:
print(f'Accuracy: {accuracy_score(y, np.array([0 for _ in range(len(y))]))}')
print(f'F1-score: {f1_score(y, np.array([0 for _ in range(len(y))]))}')

Accuracy: 0.7607182343065395
F1-score: 0.0


11. Выясните, присутствуют ли в данных пропуски. Если присутствуют, заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [ ]:
x.info() # пропусков не обнаружено

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 1 to 48842
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       48842 non-null  object
 1   1       48842 non-null  object
 2   2       48842 non-null  object
 3   3       48842 non-null  object
 4   4       48842 non-null  object
 5   5       48842 non-null  object
 6   6       48842 non-null  object
 7   7       48842 non-null  object
 8   8       48842 non-null  object
 9   9       48842 non-null  object
 10  10      48842 non-null  object
 11  11      48842 non-null  object
 12  12      48842 non-null  object
dtypes: object(13)
memory usage: 4.8+ MB


12. Выберите колонки с числовыми и категориальными переменными (используя возможности pandas).

In [5]:
x_nums = x.select_dtypes(include=['int64']).columns.tolist()
x_cats = x.select_dtypes(include=['object']).columns.tolist()
ct = ColumnTransformer([('scale',  MinMaxScaler(), x_nums),('onehot', OneHotEncoder(handle_unknown='ignore'),  x_cats)])
clfs = {'Logistic Regression': LogisticRegression(random_state=rs),'KNeighborsClassifier': KNeighborsClassifier(),'Linear SVC': LinearSVC(random_state=rs)}

13. Создайте пайплайн по обработке числовых и категориальных значений колонок (используйте OneHotEncoder,MinMaxScaler) и посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [ ]:
for clf_name, clf in clfs.items():
    result = cross_validate(estimator=Pipeline(steps=[('prepare_columns', ct),('classifier', clf)]),X=x, y=y, cv=5,scoring = ['accuracy', 'f1'], error_score="raise")
    print(clf_name)
    print(f"Accuracy: {np.mean(result['test_accuracy'])}")
    print(f"F1-score: {np.mean(result['test_f1'])}\n")

Logistic Regression
Accuracy: 0.8608370621805979
F1-score: 0.6805879516575764

KNeighborsClassifier
Accuracy: 0.8235739082217739
F1-score: 0.5984111329448487

Linear SVC
Accuracy: 0.8445600915966358
F1-score: 0.6568239367128303



14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями, (испольуйте SimpleImputer). Посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [ ]:
fill_missing = SimpleImputer(missing_values='?', strategy='most_frequent')
for clf_name, clf in clfs.items():
    result = cross_validate(estimator=Pipeline(steps=[('fill_missing', fill_missing),('prepare_columns', ct),('classifier', clf)]),
        X=x, y=y, cv=5,scoring = ['accuracy', 'f1'])
    print(clf_name)
    print(f"Accuracy: {np.mean(result['test_accuracy'])}")
    print(f"F1-score: {np.mean(result['test_f1'])}\n")

Logistic Regression
Accuracy: 0.8602228220459359
F1-score: 0.678167426230144

KNeighborsClassifier
Accuracy: 0.8233077455311053
F1-score: 0.5983947385898493

Linear SVC
Accuracy: 0.8434954198747832
F1-score: 0.6537475652226427



15. Посчитайте cross_val_score по тем же алгоритмам и метрикам, если просто удалить значения '?'.

In [ ]:
data_drop = data.replace({'?': None}).dropna()
x_drop = data_drop.loc[:,:13].iloc[1:]
y_drop = data_drop[14].map({'<=50K': 0, '>50K': 1}).iloc[1:]

for clf_name, clf in clfs.items():
    result = cross_validate(estimator=Pipeline(steps=[('prepare_columns', ct),('classifier', clf)]),X=x_drop, y=y_drop, cv=5,scoring = ['accuracy', 'f1'])
    print(clf_name)
    print(f"Accuracy: {np.mean(result['test_accuracy'])}")
    print(f"F1-score: {np.mean(result['test_f1'])}\n")

Logistic Regression
Accuracy: 0.8566406822832127
F1-score: 0.6838057293158559

KNeighborsClassifier
Accuracy: 0.8205297068639797
F1-score: 0.6057083268824883

Linear SVC
Accuracy: 0.8400557779190929
F1-score: 0.6605496132055083



 16. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier на данных с замененными значениями '?' на самые частые значения.

In [ ]:
ensembles = {'Random Forest Classifier': RandomForestClassifier(random_state=rs),'Gradient Boosting Classifier': GradientBoostingClassifier(random_state=rs)}
for clf_name, clf in ensembles.items():
    result = cross_validate(estimator=Pipeline(steps=[('prepare_columns', ct),('classifier', clf)]),X=x, y=y, cv=5,scoring = ['accuracy', 'f1'])
    print(clf_name)
    print(f"Accuracy: {np.mean(result['test_accuracy'])}")
    print(f"F1-score: {np.mean(result['test_f1'])}\n")

Random Forest Classifier
Accuracy: 0.8477744602194391
F1-score: 0.6374102565293243

Gradient Boosting Classifier
Accuracy: 0.8527702352684438
F1-score: 0.6431765543110323



17. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [6]:
ct_0 = ColumnTransformer([('onehot', OneHotEncoder(),  x_cats)])

ct_1 = ColumnTransformer([('scale',  MinMaxScaler(), x_nums),('onehot', OneHotEncoder(handle_unknown='ignore'),  x_cats)])

ct_2 = ColumnTransformer([('scale',  StandardScaler(), x_nums),('onehot', OneHotEncoder(handle_unknown='ignore'),  x_cats)])

pipe = Pipeline(steps=[('fill_missing', SimpleImputer(missing_values='?', strategy='most_frequent')),('prepare_columns', 'passthrough'),('clf', 'passthrough')])

params = [{'prepare_columns': [ct_1, ct_2],
          'clf': [RandomForestClassifier(random_state=rs),GradientBoostingClassifier(random_state=rs),
                    LogisticRegression(random_state=rs),LinearSVC(random_state=rs)]}]

search = GridSearchCV(pipe, params, scoring=['accuracy', 'f1'], refit='f1', n_jobs=-1)
search.fit(x, y)
print(search.best_params_)

{'clf': LogisticRegression(random_state=42), 'prepare_columns': ColumnTransformer(transformers=[('scale', MinMaxScaler(), []),
                                ('onehot',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                  13])])}


In [9]:
search.best_score_

0.7011781205746347